In [61]:
from preprocessing import * 
from sklearn import svm
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.metrics import cohen_kappa_score
from sklearn import pipeline

from scipy.signal import butter, sosfilt, lfilter

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [62]:
def filter(signal, band, fs):
  sos = butter(10, band, 'bandpass', fs=fs, output='sos')
  filtered = sosfilt(sos, signal)
  return filtered

In [63]:
filename = "data/A01T.gdf"
data = mne.io.read_raw_gdf(filename)


Extracting EDF parameters from d:\Study\GP\EEG-Motor-Imagery-classifier\data\A01T.gdf...
GDF file detected
Setting channel info structure...
Could not determine channel type of the following channels, they will be set as EEG:
EEG-Fz, EEG, EEG, EEG, EEG, EEG, EEG, EEG-C3, EEG, EEG-Cz, EEG, EEG-C4, EEG, EEG, EEG, EEG, EEG, EEG, EEG, EEG-Pz, EEG, EEG, EOG-left, EOG-central, EOG-right
Creating raw.info structure...


c:\Users\Mohamad Abo Bakr\AppData\Local\Programs\Python\Python39\lib\site-packages\mne\io\edf\edf.py:1123: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  etmode = np.fromstring(etmode, UINT8).tolist()[0]
c:\Users\Mohamad Abo Bakr\AppData\Local\Programs\Python\Python39\lib\contextlib.py:124: RuntimeWarning: Channel names are not unique, found duplicates for: {'EEG'}. Applying running numbers for duplicates.
  next(self.gen)


In [64]:
dataframe = data.to_data_frame()
dataframe = dataframe.drop(columns="time")


In [65]:
values = dataframe.values.T

filtered_eeg = np.zeros((9, *values.shape))

for i in range(1, 10):
  filtered_eeg[i-1] = filter(values, [4 * i, 4 * i + 4], 250)
  print(4*i, 4*i+4)




4 8
8 12
12 16
16 20
20 24
24 28
28 32
32 36
36 40


In [66]:
filtered_eeg = np.swapaxes(filtered_eeg, 1, 2)
print(filtered_eeg.shape)

(9, 672528, 25)


In [70]:
  # Get the events 
  events = mne.events_from_annotations(data)
  codes = events[1]
  events = events[0]

  # convert annotations to mne codes
  cfilter = np.asarray(['769', '770', '771', '772'])
  lis = np.asarray([codes[i] for i in cfilter])

  # filter for classes
  ev = events[np.in1d(events[:, 2], lis)]

  # extract the samples from events
  x = np.zeros((288, 9, 313, 25))
  y = np.zeros(288)

  values = dataframe.values
  
  for point in range(len(ev)):
    for i in range(9):
      x[point, i] = filtered_eeg[i, ev[point][0]:ev[point][0]+313]
      y[point] = ev[point][2] - lis[0] + 1
      # x[point] = values[ev[point][0]:ev[point][0]+313]
      # y[point] = ev[point][2] - lis[0] + 1
    
  # Create directory for numpy data
  if not os.path.exists("numpy_data"):
    os.mkdir(os.path.join(os.getcwd(), "numpy_data"))

  # get file name without extension or path
  new_name = os.path.splitext(os.path.basename(filename))[0]
  
  # Save data to numpy arrays
  np.save(f"numpy_data2/{new_name}X", x)
  np.save(f"numpy_data2/{new_name}Y", y)

Used Annotations descriptions: ['1023', '1072', '276', '277', '32766', '768', '769', '770', '771', '772']


In [99]:
x = np.load(f"numpy_data2/{new_name}X.npy")
x = np.swapaxes(x, 2, 3)
y = np.load(f"numpy_data2/{new_name}Y.npy")
print(x.shape, y.shape)

(288, 9, 25, 313) (288,)


In [100]:
i = 1

X_train, X_test, y_train, y_test = train_test_split(x, y, random_state = 100, test_size = 0.2)

print("final shapes are: ", X_train.shape, X_test.shape, y_train.shape, y_test.shape)


final shapes are:  (230, 9, 25, 313) (58, 9, 25, 313) (230,) (58,)


In [103]:
classifiers = []
for i in range(9):
  train_coeff = featurize(X_train[:, i, :, :])
  coeff_len = len(train_coeff)

  csp = [mne.decoding.CSP(8) for _ in range(coeff_len)]
  X_train_f = np.concatenate(tuple(csp[x].fit_transform(train_coeff[x], y_train) for x  in range(coeff_len)),axis=-1)

  test_coeff = featurize(X_test[:, i, :, :])
  X_test_f = np.concatenate(tuple(csp[x].transform(test_coeff[x]) for x  in range(coeff_len)),axis=-1)
  
  clf = pipeline.make_pipeline(StandardScaler(), svm.SVC())
  clf.fit(X_train_f, y_train)

  classifiers.append(clf)
  

Computing rank from data with rank=None
    Using tolerance 17 (2.2e-16 eps * 25 dim * 3.1e+15  max singular value)
    Estimated rank (mag): 25
    MAG: rank 25 computed from 25 data channels with 0 projectors
Reducing data rank from 25 -> 25
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 16 (2.2e-16 eps * 25 dim * 2.8e+15  max singular value)
    Estimated rank (mag): 25
    MAG: rank 25 computed from 25 data channels with 0 projectors
Reducing data rank from 25 -> 25
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 19 (2.2e-16 eps * 25 dim * 3.5e+15  max singular value)
    Estimated rank (mag): 25
    MAG: rank 25 computed from 25 data channels with 0 projectors
Reducing data rank from 25 -> 25
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 18 (2.2e-16 eps * 25 dim * 3.3e+15  max singular value)
    Estimated rank (

c:\Users\Mohamad Abo Bakr\AppData\Local\Programs\Python\Python39\lib\site-packages\pywt\_multilevel.py:43: UserWarning: Level value of 7 is too high: all coefficients will experience boundary effects.
  warnings.warn(


Computing rank from data with rank=None
    Using tolerance 7.6 (2.2e-16 eps * 25 dim * 1.4e+15  max singular value)
    Estimated rank (mag): 25
    MAG: rank 25 computed from 25 data channels with 0 projectors
Reducing data rank from 25 -> 25
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 6.2 (2.2e-16 eps * 25 dim * 1.1e+15  max singular value)
    Estimated rank (mag): 25
    MAG: rank 25 computed from 25 data channels with 0 projectors
Reducing data rank from 25 -> 25
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 6.7 (2.2e-16 eps * 25 dim * 1.2e+15  max singular value)
    Estimated rank (mag): 25
    MAG: rank 25 computed from 25 data channels with 0 projectors
Reducing data rank from 25 -> 25
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 6.8 (2.2e-16 eps * 25 dim * 1.2e+15  max singular value)
    Estimated ra

c:\Users\Mohamad Abo Bakr\AppData\Local\Programs\Python\Python39\lib\site-packages\pywt\_multilevel.py:43: UserWarning: Level value of 7 is too high: all coefficients will experience boundary effects.
  warnings.warn(


Computing rank from data with rank=None
    Using tolerance 5.8 (2.2e-16 eps * 25 dim * 1e+15  max singular value)
    Estimated rank (mag): 25
    MAG: rank 25 computed from 25 data channels with 0 projectors
Reducing data rank from 25 -> 25
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 4.5 (2.2e-16 eps * 25 dim * 8.2e+14  max singular value)
    Estimated rank (mag): 25
    MAG: rank 25 computed from 25 data channels with 0 projectors
Reducing data rank from 25 -> 25
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 5.6 (2.2e-16 eps * 25 dim * 1e+15  max singular value)
    Estimated rank (mag): 25
    MAG: rank 25 computed from 25 data channels with 0 projectors
Reducing data rank from 25 -> 25
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 4.8 (2.2e-16 eps * 25 dim * 8.6e+14  max singular value)
    Estimated rank (

...

Computing rank from data with rank=None
    Using tolerance 1.1 (2.2e-16 eps * 25 dim * 2e+14  max singular value)
    Estimated rank (mag): 25
    MAG: rank 25 computed from 25 data channels with 0 projectors
Reducing data rank from 25 -> 25
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 1.2 (2.2e-16 eps * 25 dim * 2.2e+14  max singular value)
    Estimated rank (mag): 25
    MAG: rank 25 computed from 25 data channels with 0 projectors
Reducing data rank from 25 -> 25
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 1.2 (2.2e-16 eps * 25 dim * 2.1e+14  max singular value)
    Estimated rank (mag): 25
    MAG: rank 25 computed from 25 data channels with 0 projectors
Reducing data rank from 25 -> 25
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 0.96 (2.2e-16 eps * 25 dim * 1.7e+14  max singular value)
    Estimated ran

c:\Users\Mohamad Abo Bakr\AppData\Local\Programs\Python\Python39\lib\site-packages\pywt\_multilevel.py:43: UserWarning: Level value of 7 is too high: all coefficients will experience boundary effects.
  warnings.warn(


Computing rank from data with rank=None
    Using tolerance 1.1 (2.2e-16 eps * 25 dim * 2.1e+14  max singular value)
    Estimated rank (mag): 25
    MAG: rank 25 computed from 25 data channels with 0 projectors
Reducing data rank from 25 -> 25
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 1.1 (2.2e-16 eps * 25 dim * 2e+14  max singular value)
    Estimated rank (mag): 25
    MAG: rank 25 computed from 25 data channels with 0 projectors
Reducing data rank from 25 -> 25
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 1.2 (2.2e-16 eps * 25 dim * 2.1e+14  max singular value)
    Estimated rank (mag): 25
    MAG: rank 25 computed from 25 data channels with 0 projectors
Reducing data rank from 25 -> 25
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 1 (2.2e-16 eps * 25 dim * 1.9e+14  max singular value)
    Estimated rank (

c:\Users\Mohamad Abo Bakr\AppData\Local\Programs\Python\Python39\lib\site-packages\pywt\_multilevel.py:43: UserWarning: Level value of 7 is too high: all coefficients will experience boundary effects.
  warnings.warn(


Computing rank from data with rank=None
    Using tolerance 1 (2.2e-16 eps * 25 dim * 1.8e+14  max singular value)
    Estimated rank (mag): 25
    MAG: rank 25 computed from 25 data channels with 0 projectors
Reducing data rank from 25 -> 25
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 1.1 (2.2e-16 eps * 25 dim * 2e+14  max singular value)
    Estimated rank (mag): 25
    MAG: rank 25 computed from 25 data channels with 0 projectors
Reducing data rank from 25 -> 25
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 0.95 (2.2e-16 eps * 25 dim * 1.7e+14  max singular value)
    Estimated rank (mag): 25
    MAG: rank 25 computed from 25 data channels with 0 projectors
Reducing data rank from 25 -> 25
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 0.95 (2.2e-16 eps * 25 dim * 1.7e+14  max singular value)
    Estimated rank

0.3173913043478261
